In [1]:
config_list = [
{
    "model": "abc",  
    "base_url": "https://123.openai.azure.com/", 
    "api_type": "azure", 
    "api_version": "2024-02-15-preview", 
    "api_key": "9r428a6b89134ffbkr562603a5e20904"
}
]

In [2]:
import autogen

In [3]:
llm_config = {
"timeout": 600,
"cache_seed": 44,  # change the seed for different trials
"config_list": config_list,
"temperature": 0,
"cache_seed": None,
}

In [4]:
RoleGenerator = autogen.AssistantAgent(
    name="RoleGenerator",
    llm_config=llm_config,
    system_message="""An experienced Role Generator. For given task, You have to list top 3 similar user roles who are in the need of the same task.
    Core Actions: 
    1. User_roles = Get_roles(rough_idea_of_requirements)
    e.g User_roles: house wife, Social Media Influencer etc""",
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
experienceduser = autogen.AssistantAgent(
    name="experienceduser",
    human_input_mode="NEVER",
    system_message="""User to Express the sample user_stories for the "rough_idea_of_requirements" for given list of User_roles. 
    Core Actions: 
    1. user_stories = SpeakUserStories(rough_idea_of_requirements, User_roles)
    """,
    code_execution_config=False,
)

In [6]:
Answerexpert = autogen.AssistantAgent(
    name="Answerexpert",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""Answer expert. Generate realistic answer for the requirement_question and rough_idea_of_requirements.
    Core Actions: 
    1. requirement_answer = AnswerQuestion(rough_idea_of_requirements, requirement_question)""",
    code_execution_config=False,
)

In [7]:
Collector = autogen.AssistantAgent(
    name="Collector",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""An experienced requirement collector. 
    Interact with the experienceduser for user_stories, and Answerexpert for requirement_answer, before proceeding.
    Core Actions: 
    1. requirement_question = ProposeQuestion(rough_idea_of_requirements, user_stories)
    2. requirement_draft = WriteReqDraf(rough_idea_of_requirements, user_stories, requirement_question, requirement_answer, draft template)""",
    code_execution_config=False,
)

In [8]:
Modeler = autogen.AssistantAgent(
    name="Modeler",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    system_message="""An experienced requirement modeler. extract including entities and relations.
    modeling_entities = ExtractEntity(requirement_draft)
    modeling_relationship = ExtractRelation(requirement_draft,modeling_entities)
    once you have done, call Documenter with modeling_entities, modeling_relationship, requirement_draft and SRS_template.
    This is SRS_template.
    ``````
    1. Introduction
               1.1 Purpose
               1.2 Intended Audience
               1.3 Intended Use
               1.4 Product Scope
               1.5 Definitions and Acronyms
    2. Overall Description
               2.1 User Needs
               2.2 Assumptions and Dependencies
    3. System Features and Requirements
                3.1 Functional Requirements
                3.2 External Interface Requirements
                3.3 System Features
                3.4 Nonfunctional Requirements
    ``````

    """,
)

In [9]:
Documenter = autogen.AssistantAgent(
    name="Documenter",
    code_execution_config=False,
    system_message="""An experienced requirement document writer. Now software_requirement_specification once you have got the Extracted Entities,  Extracted relationships, requirement_draft and SRS_template.
    This is the final step. Just display the written software_requirement_specification then TERMINATE
    """,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    human_input_mode="NEVER",
)

In [10]:
groupchat = autogen.GroupChat(
    agents=[experienceduser, RoleGenerator, Answerexpert, Collector, Modeler, Documenter], messages=[], max_round=40
)

In [11]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config,
                                   
                                  )

In [12]:
experienceduser.initiate_chat(
    manager,
    message="""A rough idea of the Requirement: 
    
    I want to develop a smart home control system.

    hint: start with RoleGenerator.
    """,
)

experienceduser (to chat_manager):

A rough idea of the Requirement: 
    
    I want to develop a smart home control system.

    hint: start with RoleGenerator.
    

--------------------------------------------------------------------------------
RoleGenerator (to chat_manager):

RoleGenerator: Develop Smart Home Control System

User_roles = Get_roles(Develop Smart Home Control System)

1. Home Automation Enthusiast - Individuals who are passionate about integrating technology into their living spaces for convenience, efficiency, and enhanced quality of life.
2. Smart Home Installers/Integrators - Professionals who specialize in setting up and configuring interconnected devices and systems in homes to create a seamless smart home experience.
3. Property Developers/Builders - Companies or individuals who construct residential properties and are looking to incorporate smart home technology to increase the value and appeal of their real estate projects.

-------------------------------